In [1]:
print("ok")

ok


In [2]:
%pwd

'g:\\Medical Bot\\End-to-End-Medical-Chatbot---Generative-AI\\research'

In [3]:
import os
os.chdir("../")

In [4]:
%pwd

'g:\\Medical Bot\\End-to-End-Medical-Chatbot---Generative-AI'

In [5]:
# Install langchain module
%pip install langchain


Note: you may need to restart the kernel to use updated packages.


In [6]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [7]:
#Extract text from a PDF file
def load_pdf_file(data):
    loader = DirectoryLoader(data,
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [8]:
extracted_data = load_pdf_file(data= 'Data/')

In [9]:
#extracted_data

In [10]:
#Split the text into chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500,
        chunk_overlap=20,
    )
    texts_chunks = text_splitter.split_documents(extracted_data)
    return texts_chunks

In [11]:
text_chunks = text_split(extracted_data)
print("Length of text chunks: ", len(text_chunks))

Length of text chunks:  5860


In [12]:
#text_chunks

In [13]:
from langchain.embeddings import HuggingFaceEmbeddings

In [14]:
# Download the Embeddings from HuggingFace
def download_huggingface_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [15]:
embeddings = download_huggingface_embeddings()

C:\Users\WALEED TRADER\AppData\Local\Temp\ipykernel_8236\2489607415.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
g:\Medical Bot\End-to-End-Medical-Chatbot---Generative-AI\MedChat\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
query_results = embeddings.embed_query("Hello world")
print("Length of query results: ", len(query_results))

Length of query results:  384


In [17]:
query_results

[-0.03447727486491203,
 0.03102317824959755,
 0.006734970025718212,
 0.026108985766768456,
 -0.03936202451586723,
 -0.16030244529247284,
 0.06692401319742203,
 -0.006441489793360233,
 -0.0474504791200161,
 0.014758856035768986,
 0.07087527960538864,
 0.05552763119339943,
 0.019193334504961967,
 -0.026251312345266342,
 -0.01010954286903143,
 -0.02694045566022396,
 0.022307461127638817,
 -0.022226648405194283,
 -0.14969263970851898,
 -0.017493007704615593,
 0.00767625542357564,
 0.05435224249958992,
 0.0032543970737606287,
 0.031725890934467316,
 -0.0846213847398758,
 -0.02940601296722889,
 0.05159561336040497,
 0.04812406003475189,
 -0.0033148222137242556,
 -0.058279167860746384,
 0.04196927323937416,
 0.022210685536265373,
 0.1281888335943222,
 -0.022338971495628357,
 -0.011656315997242928,
 0.06292839348316193,
 -0.032876335084438324,
 -0.09122604131698608,
 -0.031175347045063972,
 0.0526994913816452,
 0.04703482985496521,
 -0.08420311659574509,
 -0.030056199058890343,
 -0.02074483036

In [18]:
from dotenv import load_dotenv
load_dotenv()

True

In [19]:
PINECONE_API_KEY = os.environ.get("PINECONE_API_KEY")
# OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

In [20]:
#  Install pinecone-client module
# %pip install pinecone-client

from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medcare-medical-chatbot"

# pc.create_index(
#     name=index_name,
#     dimension=384,  # The dimension of the embeddings
#     metric="cosine",
#     spec=ServerlessSpec(
#         cloud="aws",
#         region="us-east-1",
#     )
#)

In [21]:
GROQ_API_KEY = os.environ.get("GROQ_API_KEY")
print(type(PINECONE_API_KEY), PINECONE_API_KEY)
# print(type(OPENAI_API_KEY), OPENAI_API_KEY)
print(type(GROQ_API_KEY), GROQ_API_KEY)


<class 'str'> pcsk_3TEqLd_KSZRqHyDw9phBUsmMLmUzzghGoK8k21oiY2zd9Ls7PKM4hPUU4ew9aUiP6JdQBT
<class 'str'> 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


In [22]:
import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
# os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["GROQ_API_KEY"] = GROQ_API_KEY


In [23]:
# Install langchain-pinecone if not already installed
# %pip install langchain-pinecone

# embed each chunk and upsert the embeddings into your pinecone index
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings
)

In [24]:
# Load existing Index
from langchain_pinecone import PineconeVectorStore
#Embed each chunk and upsert the embeddings into your pinecone index
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
) 

In [25]:
docsearch

In [26]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [ ]:
retrived_docs = retriever.invoke("What is the best way to treat a headache?")

In [ ]:
retrived_docs

[Document(id='5876d89f-2238-42c2-8144-646e352ddf65', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 52.0, 'page_label': '53', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Gale Encyclopedia of Medicine Vol. 1 (A-B).pdf', 'total_pages': 637.0}, page_content='with the thumb or finger, for two minutes on each arm.\n• For headaches, sinus congestion, and tension, locate the\n“GB20” points at the base of the skull in the back of the\nhead, just behind the bones in back of the ears. Disperse\nthese points for two minutes with the fingers or thumbs.\nAlso find the “yintang” point, which is in the middle of\nthe forehead between the eyebrows. Disperse it with\ngentle pressure for two minutes to clear the mind and to\nrelieve headaches.\nPrecautions'),
 Document(id='4b09b64d-928b-4bec-aca9-7ea2f7818351', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31

In [ ]:
# from langchain import OpenAI
# llm = OpenAI(temperature=0.4, max_tokens=500)

C:\Users\WALEED TRADER\AppData\Local\Temp\ipykernel_9848\2827040873.py:2: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0.4, max_tokens=500)


In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "you are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer the question. "
    "If you don't know the answer, just say that you don't know. "
    "Do not try to make up an answer. "
    "Use three sentences maximum and keep the answer concise. "
    "\n\n"
    "Context: {context}\n\n"    
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}")
    ]
)

In [ ]:
# from langchain_community.llms.openai import OpenAI
# llm = OpenAI(temperature=0.4, max_tokens=500, openai_api_key=OPENAI_API_KEY)


from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0.4,
    model_name="llama3-70b-8192",  # or "llama3-8b-8192"
    groq_api_key=os.environ["GROQ_API_KEY"]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)


In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("GROQ_API_KEY")
print(api_key)  # <-- Check if it's printing something


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


In [ ]:
# import openai
# import os

# client = openai.OpenAI(api_key=os.environ["OPENAI_API_KEY"])

# response = client.chat.completions.create(
#     model="gpt-3.5-turbo",
#     messages=[
#         {"role": "user", "content": "Hello!"}
#     ]
# )

# print(response.choices[0].message.content)



# import os
# from anthropic import Anthropic
# from dotenv import load_dotenv

# load_dotenv()

# client = Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))

# response = client.messages.create(
#     model="claude-3-sonnet-20240229",
#     max_tokens=500,
#     messages=[
#         {"role": "user", "content": "What is Acne?"}
#     ]
# )

# print(response.content[0].text)


# import os
# from dotenv import load_dotenv
# from groq import Groq

# load_dotenv()
# client = Groq(api_key=os.environ["GROQ_API_KEY"])

# response = client.chat.completions.create(
#     model="mixtral-8x7b",
#     messages=[
#         {"role": "user", "content": "What is Acne?"}
#     ]
# )

# print(response.choices[0].message.content)


# groq_chatbot.py

import os
from dotenv import load_dotenv
from groq import Groq

# Load .env file and read API key
load_dotenv()
api_key = os.getenv("GROQ_API_KEY")

if not api_key:
    raise ValueError("API key not found. Please check your .env file.")

# Initialize the Groq client
client = Groq(api_key=api_key)

# Chat request using Mixtral-8x7B
try:
    response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",  # ✅ Correct model name
        messages=[
            {"role": "user", "content": "What is Acne?"}
        ]
    )

    print("Bot:", response.choices[0].message.content)

except Exception as e:
    print("Error:", e)




Bot: **What is Acne?**

Acne is a common skin condition that occurs when the pores on the skin become clogged with oil, dead skin cells, and bacteria. It can cause a range of symptoms, including:

* Pimples or zits
* Blackheads
* Whiteheads
* Redness and inflammation
* Scarring

Acne typically affects the face, but it can also appear on the neck, back, chest, and other areas of the body. It is most common during puberty, but it can occur at any age.

**Causes of Acne:**

There are several factors that can contribute to the development of acne, including:

1. **Hormonal changes**: Fluctuations in hormone levels, such as during puberty, menstruation, or pregnancy, can lead to increased oil production and clogged pores.
2. **Overproduction of sebum**: The skin's oil glands produce sebum, which can clog pores and lead to acne.
3. **Dead skin cells**: When dead skin cells are not shed properly, they can accumulate and clog pores.
4. **Bacterial growth**: A type of bacteria called Propioniba

In [ ]:
# response = rag_chain.invoke({"input": "What is Acne?"})
# print(response['answer']) 

response = rag_chain.invoke({"input": "Your question here"})
print(response["answer"])


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}